![PyData_logo](./static/pydata-logo-madrid-2016.png)

# Remove Before Flight
## Analyzing Flight Safety Data with Python

###### Jesús Martos Carrizo
###### Alejandro Sáez Mollejo

### Loading & Cleaning data 

In [ ]:
import pandas as pd
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
# Some configuration options:
pd.set_option('max_columns', 70)
pd.set_option('max_rows', 50)

plt.style.use('ggplot')

##### A first glipse of the data: events

[Data base structure](./data/admsrel.pdf)

In [ ]:
events = pd.read_csv("./data/events.csv", sep='|', low_memory=False)
events.head()

In [ ]:
events.info(memory_usage='deep')

In [ ]:
# Parsing dates
date_format = "%m/%d/%Y %I:%M:%S %p"
date = events['ev_date'].dropna()
date = pd.to_datetime(date, format=date_format)
time = events['ev_time'].astype(str).str.split('.').str[0].str.zfill(4)
time = pd.to_datetime(time, format="%H%M", errors='coerce') # for nan values

time.fillna('0000', inplace=True)

str_date = date.dt.date.astype(str)
str_time = time.dt.time.astype(str)
date_time = pd.to_datetime(str_date + ' ' + str_time,
                           format="%Y-%m-%d %H:%M:%S")
events['ev_date'] = date_time

In [ ]:
events.info(memory_usage='deep', max_cols=1)

In [ ]:
events.columns

What we don't really need...

In [ ]:
events.drop(['ev_dow', 'ev_time', 'ev_tmzn', 'ev_year', 'ev_month'], axis=1, inplace=True)

Dropping events previuos to 1982 (They shouldn't be here!)

In [ ]:
cond = events['ev_date'].dt.year < 1982
print('Dates before 1982: ',cond.sum())
indx = events[cond].index
events.drop(indx, inplace=True)

##### Analysing *accidents*: discarding *incidents* 

In [ ]:
gby_ev_type = events.groupby('ev_type')
gby_ev_type.groups.keys()

Definitions according to _______

** ACCIDENT **

** INCIDENT**

In [ ]:
events = gby_ev_type.get_group('ACC')

In [ ]:
events['ev_id'].count()

##### Focus on commercial flights 

In [ ]:
aircraft = pd.read_csv("./data/aircraft.csv",
                                    encoding = "ISO-8859-1",  # export the table again.
                                    sep=';',
                                    low_memory=False)

In [ ]:
aircraft.head(2)

Aircrafts are classified according to different criteria (**Model,Model of the engine,far_part**, etc) in the NTSB database.

For this talk we are going to focus on **revenue commercial Air Transport**. In order to do this, we are going to filter the data according to the FAA (Federal Aviation Administration) criteria:

Code of Federal Regulations (CFR) title 14:

* **Part 121**: Large transport aircraft engaged in revenue operations involving the transport of both passengers and cargo.

* Part 135: Both scheduled (primarily passenger service) carriers flying aircraft with fewer than 10 passenger seats and on-demand passenger or cargo services using either fixed-wing airplanes or helicopters. On-demand passenger services include air taxi, air medical, and certain air tour operations.

* **Non US and commercial**

In [ ]:
events['ev_id'].count()

In [ ]:
# are accidents
cond = aircraft['ev_id'].isin(events['ev_id'])

aircraft = aircraft[cond]

In [ ]:
gby_far_part = aircraft.groupby('far_part')
gby_far_part.groups.keys()

In [ ]:
desired_far_parts = ['NUSC',  # Non-U.S. Commercial
                     '121']      # Air Carrier
                    # '129',      # Foreign
                   # '091',      # General Aviation
                 #    'NUSN',  # Non-U.S. Non-Commercial
                   #  'UNK',     # Unknown
                     #'135']

<img src="./static/pict1.png" style="width:700px;"/>

In [ ]:
cond = aircraft['far_part'].isin(desired_far_parts)
aircraft = aircraft[cond]

In [ ]:
aircraft['acft_category'].value_counts()

In [ ]:
cond_2 = aircraft['acft_category'] == 'AIR'
aircraft= aircraft[cond_2]
aircraft.head(2)

In [ ]:
ev_ids_for_desired_criteria = aircraft['ev_id'].drop_duplicates()
# How many different events do we have after this?
ev_ids_for_desired_criteria.count()

In [ ]:
#quitar columnas que no queramos 

aircraft

In [ ]:
mask = events['ev_id'].isin(ev_ids_for_desired_criteria.values)
events = events[mask]
events.head()

In [ ]:
events.info()

In [ ]:
aircraft.info()

### Data Analysis 

From now on, `events` and `aircraft` only contain **accident** data from flights corresponding to **FAR parts** in desired_far_parts.

#### Where do accidents occur?

##### What information do we have about location? 

In [ ]:
events[['ev_city', 'ev_state', 'ev_country', 'latitude', 'longitude']].head(10)

##### So, let's parse it!

In [ ]:
def convert_lat(string):
    degs = float(string[0:2])
    mins = float(string[2:4])
    secs = float(string[4:6])
    last = string[6].lower()
    if last == 's':
        factor = -1.0
    elif last == 'n':
        factor = 1.0
    else:
        raise ValueError("invalid hemisphere")
    return factor * (degs + mins / 60 + secs / 3600)

def convert_lon(string):
    degs = float(string[0:3])
    mins = float(string[3:5])
    secs = float(string[5:7])
    last = string[7].lower()
    if last == 'w':
        factor = -1.0
    elif last == 'e':
        factor = 1.0
    else:
        raise ValueError("invalid direction")
    return factor * (degs + mins / 60 + secs / 3600)

In [ ]:
# Parsing latitude
events['latitude'] = events['latitude'].replace('       ', np.nan)
lat = events['latitude']
lat.dropna(inplace=True)

mask = lat.str.contains(r'^[0-9]{6}[NnSs]$')
events['latitude_num'] = lat[mask].apply(convert_lat)

# Parsing longitude
events['longitude'] = events['longitude'].replace('        ', np.nan)
lon = events['longitude']
lon.dropna(inplace=True)

mask = lon.str.contains(r'^[0-9]{7}[EeWw]$')
events['longitude_num'] = lon[mask].apply(convert_lon)

events[['longitude_num', 'latitude_num']].head()

##### Using Google Geocode API

 ![](./static/googlemaps.png)

Due to the limitation on the daily number of requests we will load data that have requested previously.

In [ ]:
cond = events['longitude'].isnull() | events['latitude'].isnull()

events['ev_id'][cond].count()

In [ ]:
need_location = events.loc[cond][['ev_city', 'ev_country', 'ev_state', 'latitude', 'longitude']]
need_location.head()

We use `need_location.csv` and generate ---> `have_location_part1.csv` & `have_location_part_2.csv`

In [ ]:
have_location1 = pd.read_csv('./data/have_location_part1.csv', index_col='Unnamed: 0')
have_location2 = pd.read_csv('./data/have_location_part2.csv', index_col='Unnamed: 0')
have_location = pd.concat([have_location1, have_location2], axis=0)

events.loc[need_location.index, ['latitude_num']] = have_location['latitude']
events.loc[need_location.index, ['longitude_num']] = have_location['longitude']
events.loc[need_location.index, ['ev_city', 'ev_country', 'ev_state', 'latitude_num', 'longitude_num']].head()

##### and... plotting 

In [ ]:
lon_ = events['longitude_num'].values
lat_ = events['latitude_num'].values

#####  Really gathered right? but... where are the airports?

In [ ]:
header = ['name', 'city', 'country', 'lat', 'lon']
airports = pd.read_csv('./data/openflights/airports.dat', usecols=(1,2,3,6,7), names=header)
airports.head()

#### When do accidents occur? Which flight phase is safer?

<img src="./static/PED_phase_flight.jpg" style="width: 800px;"/>

<img src="./static/pict2.png" style="width:700px;"/>

In [ ]:
detailed_phase = [501,502,503,504,505,511,512,513,514,
                  521,522,523,531,541,551,552,553,
                  561,562,563,564,565,566,567,567,568,
                  569,571,572,573,574,575,576,581,582,583,
                  591,592, 542]

general_phase = [(ii//10)*10 for ii in detailed_phase[:-1]]
general_phase.append(580)

In [ ]:
aircraft['phase_gen'] = aircraft['phase_flt_spec'].replace(to_replace=detailed_phase,
                                                           value=general_phase)
aircraft['phase_gen'].value_counts()

In [ ]:
occurrences_series = aircraft['phase_gen'].value_counts()
# Only the ten most common
occurrences_series.iloc[0:10]

In [ ]:
phases_dict = {500:'STANDING',
             510:'TAXI',
             520:'TAKEOFF',
             530:'CLIMB',
             540:'CRUISE',
             550:'DESCENT',
             560:'APPROACH',
             570:'LANDING',
             580:'MANEUVERING',
#             590:'HOVER',
             600:'OTHER',
             610:'UNKNOWN'}
            
occurrences_series = occurrences_series.rename_axis(phases_dict)

In [ ]:
%matplotlib inline
occurrences_series.plot.barh(stacked=True)
plt.xlabel('number of accidents')

In [ ]:
group = aircraft.groupby(['phase_gen', 'damage'])

In [ ]:
phases_list = phases_dict.keys()
damage_list = ['NONE', 'SUBS', 'DEST', 'MINR', 'UNK']
injured = ['injured', 'fatalities']
phases = pd.DataFrame(columns=damage_list+injured)

In [ ]:
phases

In [ ]:
for phase in phases_list:
    sum_inj = 0
    sum_fat = 0
    for dam in damage_list:
        try:
            gg = group.get_group((phase, dam))
            phases.loc[phase, [dam]] = gg['ev_id'].count()
            mask = events['ev_id'].isin(gg['ev_id'])
            #print(mask.sum())
            inj_m = events[mask]['inj_tot_m'].sum()
            inj_s = events[mask]['inj_tot_s'].sum()
            fat = events[mask]['inj_tot_f'].sum()
            if not np.isnan(inj_m):
                sum_inj += inj_m
            if not np.isnan(inj_s):
                sum_inj += inj_s
            if not np.isnan(fat):
                sum_fat += fat
        except KeyError:
            pass
    
    phases.loc[phase, ['injured']] = sum_inj
    phases.loc[phase, ['fatalities']] = sum_fat
        #print(gg)
        #print(gg['ev_id'].count())
        

In [ ]:
phases.rename_axis(phases_dict, inplace=True)
phases

In [ ]:
phases['TOTAL'] = phases[['NONE', 'MINR', 'SUBS', 'DEST']].sum(axis=1)
phases = phases.sort_values(by='TOTAL', ascending=False)

In [ ]:
phases[['NONE', 'MINR', 'SUBS', 'DEST']].plot.barh(stacked=True)

In [ ]:
phases[['injured', 'fatalities']].plot.barh()

In [ ]:
phases = phases.sort_values(by='fatalities', ascending=False)
phases[['injured', 'fatalities']].plot.barh()

#### Which are the main accident causes?

In [ ]:
occurrences = pd.read_csv("./data/ocurrences.csv",sep=';')

In [ ]:
occurrences = occurrences[occurrences['ev_id'].isin(events['ev_id'])]

In [ ]:
# ocurrences_filtered['ev_id'].value_counts()
occurrences[occurrences['ev_id']=='20001214X36685']

In [ ]:
events[events['ev_id']=='20001214X36685']

In [ ]:
from IPython.display import HTML
HTML('<iframe src="http://www.ntsb.gov/_layouts/ntsb.aviation/brief.aspx?ev_id=20001214X36685&key=2&queryId=9811677a-e5d5-43d4-8d8e-89599d9d47a4&pgno=3&pgsize=50"\
      width="700" height="400"></iframe>')

In [ ]:
occurrences['Occurrence_Code'] = occurrences['Occurrence_Code'].replace(
    to_replace=[131,171,172,191,192,193,195,232,271,351,352, 353,351,354],
    value=[130,170,170,190,190,190,190,230,270,520,520,350,350,350])

In [ ]:
occurrences_series_2 = occurrences['Occurrence_Code'].value_counts()
occurrences_series_2 = occurrences_series_2[0:13]

In [ ]:
occurrences_dict = {130:'AIRFRAME/COMPONENT/SYSTEM FAILURE/MALFUNCTION',
                    170:'FIRE/EXPLOSION',
                    180:'FORCED LANDING',
                    190:'GEAR COLLAPSED',
                    200:'HARD LANDING',
                    220:'IN FLIGHT COLLISION WITH OBJECT',
                    230:'IN FLIGHT COLLISION WITH TERRAIN/WATER',
                    240:'IN FLIGHT ENCOUNTER WITH WEATHER',
                    250:'LOSS OF CONTROL - IN FLIGHT',
                    260:'LOSS OF CONTROL - ON GROUND/WATER',
                    270:'MIDAIR COLLISION',
                    280:'NEAR COLLISION BETWEEN AIRCRAFT',
                    300:'NOSE OVER',
                    310:'ON GROUND/WATER COLLISION WITH OBJECT',
                    320:'ON GROUND/WATER ENCOUNTER WITH TERRAIN/WATER',
                    340:'OVERRUN',
                    350:'LOSS OF ENGINE POWER',
                    430:'MISCELLANEOUS/OTHER',
                    520:'DESCENT'}
            
occurrences_series_2 = occurrences_series_2.rename_axis(occurrences_dict)

In [ ]:
occurrences_series_2.plot.barh(stacked=True)
plt.xlabel('number of accidents')

#### What impact does experience have?

In [ ]:
flight_crew = pd.read_csv('./data/flight_crew.csv')

# Only crew for the selected events:
flight_crew = flight_crew[flight_crew['ev_id'].isin(events['ev_id'])]
flight_crew.head(5)

In [ ]:
flight_crew['crew_category'].value_counts()

In [ ]:
%matplotlib inline
# Remove spaces
flight_crew['crew_category'] = flight_crew['crew_category'].str.strip()

# Pilots and copilots
crew_cat = ['PLT', 'CPLT']
# crew_cat = ['CPLT']
is_plt_cplt = flight_crew['crew_category'].isin(crew_cat)

In [ ]:
flight_crew_pc = flight_crew[is_plt_cplt]

flight_crew_pc['crew_age'].describe()

In [ ]:
flight_crew_pc['crew_age'].plot.box()
plt.xticks([1],['crew age'])

In [ ]:
bins = np.arange(14, 85, 5)
flight_crew_pc['crew_age'].hist(bins=bins)
plt.xlabel('crew age')

According to GAMMA databook:
    http://www.gama.aero/files/GAMA_2015_Databook_LoRes%20updated%203-29-2016.pdf
    http://www.gama.aero/media-center/industry-facts-and-statistics/statistical-databook-and-industry-outlook#year2015

In [ ]:
licenses_by_age_airline = np.array([0, 572, 5199, 12003, 15507,
                                                          18337, 23058, 25882, 24220, 16824,
                                                          10184, 4284, 1766, 787])

In [ ]:
age_group = pd.cut(flight_crew_pc['crew_age'], bins)
gby_age = flight_crew_pc['ev_id'].groupby(age_group).count()
gby_age

In [ ]:
accident_rate_age_1 = gby_age / licenses_by_age_airline
accident_rate_age_1.plot.bar()
plt.ylabel('accidents / licenses')
plt.xlabel('crew age')

It seems that some people have investigated this issue before:
>  [...] the accident rate of airline transport
rated (ATR) pilots aged 55–59 (3.78/1,000) is approximately one-third of that of pilots with the
same rating who are aged 20–24 (11.71/10,000). <cite>John A. Wise, V. David Hopkin, Daniel J. Garland, **Handbook of Aviation Human Factors**, Second Edition.</cite>

In [ ]:
df = pd.DataFrame({})
for ii, role in enumerate(['PLT', 'CPLT']):
    print(ii, role)
    is_role = flight_crew['crew_category'] == role
    flight_crew_role = flight_crew[is_role]
    
    age_group = pd.cut(flight_crew_role['crew_age'], bins)
    df[role] = flight_crew_role['ev_id'].groupby(age_group).count() / licenses_by_age_airline

df.plot.bar(stacked=True)

In [ ]:
flight_time = pd.read_csv('./data/flight_time.csv', sep=';', usecols=(0,1,2,3,4,5))
cond = flight_time['ev_id'].isin(events['ev_id'])
flight_time = flight_time[cond]

In [ ]:
flight_time['flight_type'] = flight_time['flight_type'].str.split().str[0]
flight_time['flight_craft'] = flight_time['flight_craft'].str.split().str[0]

In [ ]:
flight_type = list(flight_time.groupby('flight_type').groups.keys())
print(flight_type)

In [ ]:
flight_craft = list(flight_time.groupby('flight_craft').groups.keys())
print(flight_craft)

In [ ]:
gby_flight_time = flight_time.groupby(['flight_craft', 'flight_type'])

In [ ]:
from ipywidgets import interact

In [ ]:
def my_widget_function(fcraft, ftype, factor):
    flight_h = gby_flight_time.get_group((fcraft, ftype))['flight_hours']
    print(flight_h.describe())
    max_v = flight_h
    cond = flight_h < factor *  flight_h.max()
    
    fig, ax = plt.subplots(1, 2, figsize=(12,4))
    flight_h[cond].hist(ax=ax[0])
    ax[0].set_ylabel('flight hours')
    flight_h[cond].plot.box(ax=ax[1])

In [ ]:
interact(my_widget_function,
         fcraft=flight_craft,
         ftype=flight_type,
         factor=(0., 1., 0.05))

http://aviation.stackexchange.com/questions/1807/what-is-common-number-of-flight-hours-a-year-for-an-airline-pilot

#### And finally... is commercial air transport safer now than 30 years ago?

In [ ]:
ev_before_2016 = events[events['ev_date'].dt.year < 2016]
gby_year = events.groupby(ev_before_2016.ev_date.dt.year)

injured_per_year = gby_year[['inj_tot_f', 'inj_tot_s', 'inj_tot_m']].count()

#injured_per_year.index = np.arange(1982, 2016)
injured_per_year.head()

In [ ]:
injured_per_year.plot.area(alpha=0.8, figsize=(11,5))
plt.xlabel('year')

In [ ]:
passengers = pd.read_csv('./data/annual_passengers_carried/data.csv', nrows=1, usecols=range(4,60))
passengers = passengers.transpose()
# renaming column
passengers.columns = ['passengers']
passengers
# parsing date in index 
passengers.index = pd.to_datetime(passengers.index.str[:4])
# converting flight number to number
passengers['passengers'] = pd.to_numeric(passengers['passengers'], errors='coerce') / 1e6
passengers.index = passengers.index.year

In [ ]:
flights = pd.read_csv('./data/annual_worldwide_departures/data.csv', nrows=1, usecols=range(4,59))
flights = flights.transpose()
# renaming column
flights.columns = ['flights']
flights
# parsing date in index 
flights.index = pd.to_datetime(flights.index.str[:4])
# converting flight number to number
flights['flights'] = pd.to_numeric(flights['flights'], errors='coerce') / 1e6

flights.index = np.arange(1960,2015)

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(12,6))
ax[0].set_title('Millions of passengers transported')
passengers['passengers'].plot.area(ax=ax[0], alpha=0.6, color="#0072B2")
ax[0].set_xlim(1975, 2014)

ax[1].set_title('Number of flights (millions)')
flights['flights'].plot.area(ax=ax[1], alpha=0.6)
ax[1].set_xlim(1975, 2014)

plt.tight_layout()

In [ ]:
letality_rate = injured_per_year['inj_tot_f'] / passengers['passengers']
ax = letality_rate.plot.area(alpha=0.8, figsize=(11,5))
ax.set_xlim(1982, 2015)

In [ ]:
accidents_gby_year = events['ev_id'].groupby(events.ev_date.dt.year).count()
ax = accidents_gby_year.plot.area(alpha=0.8, figsize=(11,5))
ax.set_xlim(1982, 2015)

In [ ]:
accident_rate = accidents_gby_year / flights.flights
accident_rate.tail()

In [ ]:
ax = accident_rate.plot.area(alpha=0.8, figsize=(11,5))
ax.set_xlim(2000, 2009)

In [ ]:
ax = accident_rate.loc[2000:2010].plot.bar(figsize=(11,5))

![PyData_logo](./static/pydata-logo-madrid-2016.png)


---


In [6]:
# Notebook style
from IPython.core.display import HTML
css_file = './static/style.css'
HTML(open(css_file, "r").read())